# Imports

In [2]:
import pandas as pd
import numpy as np
import requests
from PyPDF2 import PdfFileReader, PdfFileMerger, PdfFileWriter
import PyPDF2
from bs4 import BeautifulSoup
import os
import shutil
from urllib.parse import urljoin
from urllib.request import Request, urlopen
from io import StringIO, BytesIO
import cloudinary.uploader
import json
from google.cloud import storage
import pdfminer.layout
import pdfminer.high_level
import pdfplumber
import pdfminer.pdftypes
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar, LTTextBox
from pdfminer.pdfpage import PDFPage

import re

/Users/moritzgeiger/.pyenv/versions/3.9.1/envs/pwc/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [ ]:
["Conta que permite aceder aos seguintes produtos e servi\u00e7os, mediante o pagamento de uma",           
"Conta para clientes sem cr\u00e9dito no ABANCA e com aplica\u00e7\u00f5es financeiras de valor inferior",
"Conta para clientes dos 0 aos 28 anos.",         
"Conta regulada pelo Decreto-Lei n.\u00ba 27 C/2000, de 10 de mar\u00e7o, com altera\u00e7\u00f5es posteriores."]

In [3]:
URL = 'https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf'
remote = urlopen(Request(URL)).read()
memory = BytesIO(remote)

pdf = pdfplumber.open(memory)

In [10]:
for page in pdf.pages[4: 5]: 
    prod_df = pd.DataFrame(page.extract_table())
    print(prod_df)

                                                   0   \
0                                                None   
1                                                None   
2                                                       
3                                                None   
4                                                None   
5                                                None   
6                                                None   
7                                                None   
8   1.  Emissão de extrato \n1.1  Mensal (enviado ...   
9   2.   Fotocópias de segundas vias de talões de ...   
10  3.   Manutenção de conta \n3.1 Conta D.O. Part...   
11  4.  Manutenção de Conta Pacote \n4.1. Conta Fu...   
12              5.  Manutenção de Conta Base (Nota 6)   
13  6.  Manutenção  de  Conta  Serviços  Mínimos  ...   
14  7.   Levantamento de numerário  \n 7.1 Ao balc...   
15        8.   Adesão ao serviço de banca à distância   
16  9.  Depósito de moedas metá

# Sources

In [119]:
data = {"1": {"bp_bank_id": "0170", "last_updated": "2021-04-12", "url": "https://www.abanca.pt", "bp_pdf_url": "https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf", "price_page": "https://www.abanca.pt/pt/precario/", "list_pdfs": {"urls": ["https://www.abanca.pt/files/documents/precario-folheto-comissoes-823a7663.pdf", "https://www.abanca.pt/files/documents/folheto-taxa-juro-precario-5890906f.pdf", "https://www.abanca.pt/files/documents/precariodevaloresmobiliarios-23c714a2.pdf", "https://www.abanca.pt/files/documents/glossario-6479e89f.pdf", "https://www.abanca.pt/files/docs/leaflet-your-rights-payments-eu.pdf"], "cloud_url_size": "2129289", "cloud_merged_url": "https://storage.googleapis.com/bank_price_pdfs/1_all_products_210412130040.pdf"}, "num_pdfs": 5, "status": "ok"}, "2": {"bp_bank_id": "0269", "last_updated": "2021-04-12", "url": "https://www.bankinter.pt", "bp_pdf_url": "https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf", "price_page": "https://www.bankinter.pt/precario1", "list_pdfs": {"urls": ["https://www.bankinter.pt/documents/20182/115991/Pre%25C3%25A7%25C3%25A1rio%2BCompleto%2BTFSI%2B%2B20210401.pdf/c489f0cd-3831-4441-812e-f351741d2b22", "https://banco.bankinter.pt/particulares/pdfs/precario/folha_rosto.pdf", "https://banco.bankinter.pt/particulares/pdfs/precario/informacao_geral.pdf", "https://banco.bankinter.pt/particulares/pdfs/precario/informacao_complementar.pdf", "https://banco.bankinter.pt/particulares/pdfs/precario/p_fcd_contas_deposito.pdf", "https://banco.bankinter.pt/particulares/pdfs/precario/p_fcd_operacoes_credito.pdf", "https://banco.bankinter.pt/particulares/pdfs/precario/p_fcd_cartoes_credito_debito.pdf", "https://banco.bankinter.pt/particulares/pdfs/precario/p_fcd_cheques.pdf", "https://banco.bankinter.pt/particulares/pdfs/precario/p_fcd_transferencias.pdf", "https://banco.bankinter.pt/particulares/pdfs/precario/p_fcd_cobrancas.pdf", "https://banco.bankinter.pt/particulares/pdfs/precario/p_fcd_prestacao_servicos.pdf", "https://banco.bankinter.pt/particulares/pdfs/precario/o_fcd_contas_deposito.pdf", "https://banco.bankinter.pt/particulares/pdfs/precario/o_fcd_operacoes_credito.pdf", "https://banco.bankinter.pt/particulares/pdfs/precario/o_fcd_cartoes_credito_debito.pdf", "https://banco.bankinter.pt/particulares/pdfs/precario/o_fcd_cheques.pdf", "https://banco.bankinter.pt/particulares/pdfs/precario/o_fcd_transferencias.pdf", "https://banco.bankinter.pt/particulares/pdfs/precario/o_fcd_cobrancas.pdf", "https://banco.bankinter.pt/particulares/pdfs/precario/o_fcd_prestacao_servicos.pdf", "https://banco.bankinter.pt/particulares/pdfs/precario/o_fcd_operacoes_estrangeiro.pdf", "https://banco.bankinter.pt/particulares/pdfs/precario/p_ftj_contas_deposito.pdf", "https://banco.bankinter.pt/particulares/pdfs/precario/p_ftj_operacoes_credito.pdf", "https://banco.bankinter.pt/particulares/pdfs/precario/o_ftj_contas_deposito.pdf", "https://banco.bankinter.pt/particulares/pdfs/precario/o_ftj_operacoes_credito.pdf", "https://banco.bankinter.pt/particulares/pdfs/precario/ptfs_a.pdf", "https://banco.bankinter.pt/particulares/pdfs/precario/ptfs_b.pdf", "https://banco.bankinter.pt/particulares/pdfs/precario/ptfs_c.pdf", "https://banco.bankinter.pt/particulares/pdfs/precario/ptfs_d.pdf"], "cloud_url_size": "89399717", "cloud_merged_url": "https://storage.googleapis.com/bank_price_pdfs/2_all_products_210412130350.pdf"}, "num_pdfs": 27, "status": "ok"}, "3": {"bp_bank_id": "0079", "last_updated": "2021-04-12", "url": "https://www.bancobic.ao", "bp_pdf_url": "https://clientebancario.bportugal.pt/sites/default/files/precario/0079_/0079_PRE_0.pdf", "price_page": "https://www.bancobic.ao/inicio/precario", "list_pdfs": {"urls": ["https://www.bancobic.ao/dotAsset/0f07d8a4-4ec3-4111-a453-c4ee14d5c66c.pdf", "https://www.bancobic.ao/dotAsset/7e1e6dd2-9d56-43ef-b398-ac3bac3cc70e.pdf"], "cloud_url_size": "2865533", "cloud_merged_url": "https://storage.googleapis.com/bank_price_pdfs/3_all_products_210412130840.pdf"}, "num_pdfs": 2, "status": "ok"}, "4": {"bp_bank_id": "0008", "last_updated": "2021-04-12", "url": "https://www.bancobai.ao", "bp_pdf_url": "https://clientebancario.bportugal.pt/sites/default/files/precario/0008_/0008_PRE.pdf", "price_page": "https://www.bancobai.ao/pt/pre%C3%A7%C3%A1rio", "list_pdfs": {"urls": ["https://www.bancobai.ao/media/2988/bai_pre_clientes-particulares_31-03-2021.pdf", "https://www.bancobai.ao/media/2989/bai_pre_outros-clientes_31-03-2021.pdf", "https://www.bancobai.ao/media/2823/bai_resumo-das-alteracoes-do-precario-bna-8-02-2021-cleaned.pdf", "https://www.bancobai.ao/media/2278/termos-e-condi%C3%A7%C3%B5es.pdf"], "cloud_url_size": "6081979", "cloud_merged_url": "https://storage.googleapis.com/bank_price_pdfs/4_all_products_210412130952.pdf"}, "num_pdfs": 4, "status": "ok"}, "5": {"bp_bank_id": "0193", "last_updated": "2021-04-12", "url": "https://www.bancoctt.pt/", "bp_pdf_url": "https://clientebancario.bportugal.pt/sites/default/files/precario/0193_/0193_PRE.pdf", "price_page": "https://www.bancoctt.pt/application/themes/pdfs/precario.pdf?language_id%3D1555597541833", "list_pdfs": {"urls": ["https://www.bancoctt.pt/application/themes/pdfs/precario.pdf?language_id%3D1555597541833"], "cloud_url_size": "2216262", "cloud_merged_url": "https://storage.googleapis.com/bank_price_pdfs/5_all_products_210412131014.pdf"}, "num_pdfs": 1, "status": "ok"}}

In [3]:
for val in data.values():
    print(val.get('url'))
    print(val.get('bp_pdf_url'))
    print(f"{val.get('list_pdfs').get('cloud_merged_url')}\n")

https://www.abanca.pt
https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf
https://storage.googleapis.com/bank_price_pdfs/1_all_products_210412130040.pdf

https://www.bankinter.pt
https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf
https://storage.googleapis.com/bank_price_pdfs/2_all_products_210412130350.pdf

https://www.bancobic.ao
https://clientebancario.bportugal.pt/sites/default/files/precario/0079_/0079_PRE_0.pdf
https://storage.googleapis.com/bank_price_pdfs/3_all_products_210412130840.pdf

https://www.bancobai.ao
https://clientebancario.bportugal.pt/sites/default/files/precario/0008_/0008_PRE.pdf
https://storage.googleapis.com/bank_price_pdfs/4_all_products_210412130952.pdf

https://www.bancoctt.pt/
https://clientebancario.bportugal.pt/sites/default/files/precario/0193_/0193_PRE.pdf
https://storage.googleapis.com/bank_price_pdfs/5_all_products_210412131014.pdf



# Finding page on BP PDF

In [120]:
request = { 
    '1': {
              'url': 'www.abanca.pt',
              'bp_pdf_url': 'https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf',
              'cloud_merged_url': 'https://storage.googleapis.com/bank_price_pdfs/1_all_products_210412130040.pdf',
              'products':
                            {
                              'demand_deposit': {'withdrawal':['Emissão de extrato', 'Extrato Integrado', 'Extrato Mensal'],
                                                'documents_copy':['Fotocópias de segundas vias de talões de depósito',
                                                                'Emissão 2ªs Vias de Avisos e Outros Documentos', 'Extracto avulso',
                                                                'Segundas vias (pedido na agência)'],
                                                'portuguese':'Contas de Depósito'
                                                },
                              'housing_credit': {'comm_hash_2':'lala', 
                                                'portuguese':'Operações de crédito'}
                            }
            },
    '2': {
              'url': 'https://www.bankinter.pt',
              'bp_pdf_url': 'https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf',
              'cloud_merged_url': 'https://storage.googleapis.com/bank_price_pdfs/2_all_products_210412130350.pdf',
              'products':
                           {
                              'demand_deposit': {'withdrawal':['Emissão de extrato', 'Extrato Integrado', 'Extrato Mensal'],
                                                'documents_copy':['Fotocópias de segundas vias de talões de depósito',
                                                                'Emissão 2ªs Vias de Avisos e Outros Documentos', 'Extracto avulso',
                                                                'Segundas vias (pedido na agência)'],
                                                'portuguese':'Contas de Depósito'
                                                },
                              'housing_credit': {'comm_hash_2':'lala', 
                                                'portuguese':'Operações de crédito'}
                            }
            }
             }

In [121]:
string = 'Operações de crédito / Particulares – Pág. 1 /15'
for i, val in request.items():
    for k, v in val.items():
        print(k)
        ## opening the bp file
#     looking for the products and their pt names
    products = val.get('products')
    for product, val in products.items():
        val['pages'] = []
#     pt_terms = {key:val.get('portuguese').lower() for (key, val) in products.items()}
#     pt_terms_re = {key:re.sub('[^a-zA-Z0-9 \n\.]|\sde\s', '.{,5}', val)+'.{,5}particulares.{,5}p[áa]g' for (key,val) in pt_terms.items()}
#     for product, term in pt_terms_re.items():
#         print(f"r'{term}'")
#         print(string.lower())
#         if re.search(term, string.lower()):
#             print('found')
#             products[product]['pages'].append('yesso')
products

url
bp_pdf_url
cloud_merged_url
products
url
bp_pdf_url
cloud_merged_url
products


{'demand_deposit': {'withdrawal': ['Emissão de extrato',
   'Extrato Integrado',
   'Extrato Mensal'],
  'documents_copy': ['Fotocópias de segundas vias de talões de depósito',
   'Emissão 2ªs Vias de Avisos e Outros Documentos',
   'Extracto avulso',
   'Segundas vias (pedido na agência)'],
  'portuguese': 'Contas de Depósito',
  'pages': []},
 'housing_credit': {'comm_hash_2': 'lala',
  'portuguese': 'Operações de crédito',
  'pages': []}}

In [112]:
single_bank = {
              'url': 'www.abanca.pt',
              'bp_pdf_url': 'https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf',
              'cloud_merged_url': 'https://storage.googleapis.com/bank_price_pdfs/1_all_products_210412130040.pdf',
              'products':
                            {
                              'demand_deposit': {'withdrawal':['Emissão de extrato', 'Extrato Integrado', 'Extrato Mensal'],
                                                'documents_copy':['Fotocópias de segundas vias de talões de depósito',
                                                                'Emissão 2ªs Vias de Avisos e Outros Documentos', 'Extracto avulso',
                                                                'Segundas vias (pedido na agência)'],
                                                'portuguese':'Contas de Depósito'
                                                },
                              'housing_credit': {'comm_hash_2':'lala', 
                                                'portuguese':'Operações de crédito'}
                            }
            }

In [114]:
single_bank.get('bp_pdf_url')


'https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf'

In [111]:
for i, val in request.items():
    ## opening the bp file
    url = val.get('bp_pdf_url')
    remote = urlopen(Request(url)).read()
    memory = BytesIO(remote)
    pdf = pdfplumber.open(memory)
    # looking for the products
    products = val.get('products')
    # initializing 'pages' list inside products
    for product, val in products.items():
        val['pages'] = []
    # avoiding loading time => looking for all product terms on each page instead of one product per loop
    pt_terms = {key:val.get('portuguese').lower() for (key, val) in products.items()}
    pt_terms_re = {key:re.sub('[^a-zA-Z0-9 \n\.]|\sde\s', '.{,5}', val)+'.{,5}particulares.{,5}p[áa]g' for (key,val) in pt_terms.items()}
    
    for page in pdf.pages:
        text = page.extract_text().lower()
        for product, term in pt_terms_re.items():
            if re.search(term, text):
                products[product]['pages'].append(page.page_number)
products

{'demand_deposit': {'withdrawal': ['Emissão de extrato',
   'Extrato Integrado',
   'Extrato Mensal'],
  'documents_copy': ['Fotocópias de segundas vias de talões de depósito',
   'Emissão 2ªs Vias de Avisos e Outros Documentos',
   'Extracto avulso',
   'Segundas vias (pedido na agência)'],
  'portuguese': 'Contas de Depósito',
  'pages': [5, 6, 7, 8]},
 'housing_credit': {'comm_hash_2': 'lala',
  'portuguese': 'Operações de crédito',
  'pages': [9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40]}}

In [4]:
searchterms = {'Demand Deposits':r'[cC]ontas\sde\s[dD]ep[oó]sito.+articulares.+P[áa]g',
            'Housing Credit':r"[Oo]pera[çc][õo]es.{,4}[Cc]r[ée]dito.+articulares.+P[áa]g"
           }

# Operações de crédito / Particulares – Pág. 1 /15 
# Operações de Crédito / Particulares - Página 1 / 32
# Operações de Crédito / Particulares - Pág. 1 /29
# Operações Crédito-Particulares - Pág.1/2

In [5]:
def page_finder(searchterms):
    products = {}
    for k, val in data.items():
        url = val.get('bp_pdf_url')
        remote = urlopen(Request(url)).read()
        memory = BytesIO(remote)

        pdf = pdfplumber.open(memory)
        products[k] = {'Demand Deposits':[],
                      'Housing Credit':[]}
        for page in pdf.pages:
            if re.search(searchterms.get('Demand Deposits'), page.extract_text()):
                products[k]['Demand Deposits'].append(page.page_number)
            if re.search(searchterms.get('Housing Credit'), page.extract_text()):
                products[k]['Housing Credit'].append(page.page_number)

    return products

{'1': {'Demand Deposits': [5, 6],
  'Housing Credit': [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]},
 '2': {'Demand Deposits': [5, 6, 7, 8],
  'Housing Credit': [9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40]},
 '3': {'Demand Deposits': [5, 6, 7],
  'Housing Credit': [8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36]},
 '4': {'Demand Deposits': [5, 6], 'Housing Credit': [7, 8]},
 '5': {'Demand Deposits': [5, 6], 'Housing Credit': [7, 8]}}